In [1]:

import pandas as pd

In [2]:

df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:

df.head(10)

,artist,song,text
0,Jose Mari Chan,Can't We Start Over Again,It's been a long long while \r\nI know I've c...
1,Christy Moore,Dunnes Stores,Close your eyes and come with me back to 1984 ...
2,Eurythmics,Beethoven,Take a girl like that \r\nAnd put her in a na...
3,David Guetta,Baby When The Light,In spite of all the consequence \r\nIn spite ...
4,Unwritten Law,The Legend Of Johnny And Sarah,Johnny \r\nWas just a kid I knew \r\nAnd may...
5,Rihanna,Close To You,"[Verse 1] \r\nNothing but a tear, that's all ..."
6,David Bowie,Lady Grinning Soul,"She'll come, she'll go. She'll lay belief on y..."
7,Air Supply,A Little Bit More,"For certain she was the face of springtime, \..."
8,Lil Wayne,I Miss My Dawgs,"Yea, yea, yea, yea \r\nThis is the Carter mot..."
9,Mc Hammer,Break 'em Off Somethin' Proper,Chorus (8x): (Q-tip) \r\nIf you say Hammer Pr...


In [9]:

df['text'][0]

"It's been a long long while  \r\nI know I've caused you pain  \r\nLiving without you it's not quite the same  \r\nCan we start over again  \r\n  \r\nI played around to long  \r\nRunning from end to end  \r\nTook you for granted I've been such a fool  \r\nCan't we start over again  \r\n  \r\nAll through my life of seasons  \r\nNo matter how far I long  \r\nAlways there's something missing inside  \r\nYearning to come back home  \r\n  \r\nYou are the one I return to  \r\nLove only love leads the way  \r\nGive us this chance to start over again  \r\nDarling I'm coming home to stay  \r\n  \r\nYou are the one I return to  \r\nLove only love leads the way  \r\nGive us this chance to start over again  \r\nDarling I'm coming home to stay  \r\n  \r\nIt's been a long long while  \r\nI know I've caused you pain  \r\nLiving without you it's not quite the same  \r\nCan we start over again  \r\n  \r\nI played around to long  \r\nRunning from end to end  \r\nTook you for granted I've been such a foo

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [18]:
import nltk
nltk.download('punkt')  # You already have this
nltk.download('punkt_tab')  # This might be needed


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nirma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nirma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.0324407 , 0.05881121, ..., 0.02655737, 0.00833881,
       0.03321546])

In [23]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [25]:
# Check for case-insensitive matches
possible_matches = df[df['song'].str.lower() == "crying over you".lower()]
print(f"Found {len(possible_matches)} case-insensitive matches")

# Print a few sample songs from your dataset
print("\nSample songs in your dataset:")
print(df['song'].sample(5).tolist())

# Check if there are songs with similar names
print("\nSongs containing 'crying':")
crying_songs = df[df['song'].str.lower().str.contains('crying')]
if not crying_songs.empty:
    print(crying_songs['song'].tolist())
else:
    print("No songs with 'crying' found")

Found 0 case-insensitive matches

Sample songs in your dataset:
['Sarah', 'Revenge For Love', "It's Over Now", 'Happy Birthday', 'Wasted My Hate']

Songs containing 'crying':
["It's All Over But The Crying", 'Crying In The Chapel', 'Crying', 'Crying', 'Crying In The Chapel', 'Crying Time']


In [26]:
def recommendation(song_name):
    # First check if the song exists in our dataset
    matching_songs = df[df['song'] == song_name]
    
    if matching_songs.empty:
        return f"Song '{song_name}' not found in the dataset."
    
    # Get the index of the song
    idx = matching_songs.index[0]
    
    # Calculate similarity and get recommendations
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    # Get top 5 similar songs (excluding the input song itself)
    songs = []
    for i in range(1, min(6, len(distances))):  # Start from 1 to exclude the song itself
        songs.append(df.iloc[distances[i][0]]['song'])
    
    return songs

In [27]:
# Display the first few rows of the dataset
print("First 5 rows of the dataset:")
print(df.head())

# Show how many songs are in the dataset
print(f"\nTotal number of songs in the dataset: {len(df)}")

# Display 10 random song names from the dataset
print("\n10 random songs from the dataset:")
print(df['song'].sample(10).tolist())

First 5 rows of the dataset:
           artist                            song  \
0  Jose Mari Chan       Can't We Start Over Again   
1   Christy Moore                   Dunnes Stores   
2      Eurythmics                       Beethoven   
3    David Guetta             Baby When The Light   
4   Unwritten Law  The Legend Of Johnny And Sarah   

                                                text  
0  it 's been a long long while i know i 've caus...  
1  close your eye and come with me back to 1984 w...  
2  take a girl like that and put her in a natur s...  
3  in spite of all the consequ in spite of all my...  
4  johnni wa just a kid i knew and mayb he did n'...  

Total number of songs in the dataset: 5000

10 random songs from the dataset:
['Ebony And Ivory', 'Away From You', 'Sleigh Ride', 'A Room At The Heartbreak Hotel', 'All Is Well', 'He Thinks I Still Care', 'All Hell Breaks Loose', 'Lost', 'Smoke Signals', 'Destination']


In [28]:
 recommendation("Jose Mari Chan ")  # Replace with an actual song name from your dataset

"Song 'Jose Mari Chan ' not found in the dataset."

In [29]:
# Print the first 10 song names in your dataset
print("First 10 songs in the dataset:")
for song in df['song'].head(10):
    print(f"- {song}")

# If this doesn't show useful results, check if your column name is correct
print("\nDataframe columns:")
print(df.columns.tolist())

First 10 songs in the dataset:
- Can't We Start Over Again
- Dunnes Stores
- Beethoven
- Baby When The Light
- The Legend Of Johnny And Sarah
- Close To You
- Lady Grinning Soul
- A Little Bit More
- I Miss My Dawgs
- Break 'em Off Somethin' Proper

Dataframe columns:
['artist', 'song', 'text']


In [30]:
recommendation("Can't We Start Over Again")  # Use an exact song name from your dataset

['Come Home To Me',
 'As Long As I Love',
 'World Of Stone',
 'Today I Started Loving You Again',
 'Love Will Lead You Back']

In [31]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))